<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
## files and direcotries
kdd_dir = 'path'
file_path = '/Users/zohre/code/Jackfn96/dataset/with_annotation/videos/'

files = [f[:-4] for f in os.listdir(file_path) if os.path.isfile(os.path.join(file_path, f))]
files.sort()
print(len(files))

In [ ]:
## parameters
fps = 1/2 # 
for file_name in files:
    video_annot_df = pd.read_csv(f'{root_dir}{file_name}.mp4.csv', sep=';')
    video_annot_df.drop('Unnamed: 0', inplace=True, axis=1)
    video_annot_df['exp_id'] = file_name
    video_annot_df = timestamp_formatting(video_annot_df, 'timestamp_lena')
    video_annot_df['event_onset'] = video_annot_df['Instant']/1000
    video_annot_df['event_end'] = video_annot_df['event_onset'] + video_annot_df['duree']
    print('the shape of video annotation dataframe is', video_annot_df.shape)

    ## kss dataframe
    df = pd.read_csv(f'{root_dir}{file_name}.csv', sep=';') # temporary df to load kss file
    df.drop(columns = ['Unnamed: 0', 'Unnamed: 4'], inplace=True, axis=1)
    video_dur = df.iloc[4, :].values
    print(f'{file_name} : {video_dur}')
    column_names = df.iloc[6, :].values
    kss_raw_df = pd.DataFrame(df.iloc[7:, :].values, columns = column_names)
    kss_raw_df.drop('Buffer', inplace = True, axis=1)
    print('the shape of kss data frame is', kss_raw_df.shape)
    
    ## correction of kss_raw_df
    frame_numbers = int(video_dur[1])*60 + int(video_dur[2])
    onset= int(kss_raw_df.iloc[0,0].split('-')[0].split(':')[1])
    ## alternatively if you don't have the interval columns, you can use 
    # hand_sign = df.iloc[3, :].values
    # onset =  int(hand_sign[1])*60 + int(hand_sign[2])
    
    ## kss_interval is probably constant for all experiments ~ 300s
    kss_interval = int(kss_raw_df.iloc[0,0].split('-')[1].split(':')[0])*60
    
    
    video_timing_df = pd.DataFrame.from_records([{'duration': frame_numbers, 
                                                  'kss_onset':onset,
                                                  'kss_interval': kss_interval,
                                                  'fps': fps}])
    ## from here on you should repeat the kss accoring to the sampling rate of HRV recording,
    ## I had 2frames per seconds 
    recording_time = np.arange(0, frame_numbers, fps).tolist() 
    
    ## populating a dataframe according to the fps with corresponding kss values
    kss_impute = np.zeros(len(recording_time))
    interval = int(kss_interval / fps)
    for i in range(kss_raw_df.shape[0]):
        kss_impute[i*interval+onset: (i+1)*interval+onset] = kss_raw_df.iloc[i, 1]
    # x = downsample_to_proportion(kss_impute, .03333)
    plt.plot(kss_impute)
    
    ## merging the kss annotation with frames 
    kss_annotations = {'time_points': recording_time, 'kss_value': kss_impute }
    kss_annotations_df = pd.DataFrame.from_dict(kss_annotations)

    
    ## saving
    kss_annotations_df.to_csv(f'{kdd_dir}kss_{file_name}.csv')
